<a href="https://colab.research.google.com/github/juanfranbrv/curso-langchain/blob/main/Cadena%20sencilla%20con%20LangGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Configuración del entorno del cuaderno**



In [9]:
# Importar la librería `userdata` de Google Colab.
# Esta librería se utiliza para acceder a datos de usuario almacenados de forma segura en el entorno de Colab.
from google.colab import userdata

# Obtener las claves API de diferentes servicios desde el almacenamiento seguro de Colab.
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
# GROQ_API_KEY=userdata.get('GROQ_API_KEY')
# GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
# HUGGINGFACEHUB_API_TOKEN=userdata.get('HUGGINGFACEHUB_API_TOKEN')

# Instalar las librerías necesarias usando pip.
# El flag `-qU` instala en modo silencioso (`-q`) y actualiza las librerías si ya están instaladas (`-U`).
%pip install langchain -qU  # Instalar la librería principal de LangChain.
%pip install langgraph -qU

# Instalar las integraciones de LangChain con diferentes proveedores de LLMs.
%pip install langchain-openai -qU
# %pip install langchain-groq -qU
# %pip install langchain-google-genai -qU
# %pip install langchain-huggingface -qU

# Importar las clases necesarias de LangChain para crear plantillas de prompt.
# `ChatPromptTemplate` es la clase base para plantillas de chat.
from langchain.prompts import ChatPromptTemplate


# Importamos las clases necesarias para trabajar con cadenas
from langchain.chains import LLMChain

# Importar las clases para interactuar con los diferentes LLMs a través de LangChain.
from langchain_openai import ChatOpenAI
# from langchain_groq import ChatGroq
# from langchain_google_genai import ChatGoogleGenerativeAI
# from langchain_huggingface import HuggingFaceEndpoint

# Importamos la libreria para formatear mejor la salida
from IPython.display import Markdown, display

Recordemos aqui que los modelos de lenguaje más modernos, como **gpt-3.5-turbo** y **gpt-4**, están diseñados para funcionar de manera óptima en un formato conversacional. Esto implica que, en lugar de enviar un único "prompt de texto" como entrada, podemos organizar la información en una secuencia de mensajes con roles específicos:

-   **system**: Mensaje del sistema (define el contexto o comportamiento del asistente).
    
-   **user**: Mensaje del usuario (preguntas o instrucciones del usuario).
    
-   **assistant**: Mensaje del asistente (respuestas generadas por el modelo).
    

LangChain admite varios tipos de mensajes, incluidos `HumanMessage`, `AIMessage`, `SystemMessage` y `ToolMessage`.

Estos representan un mensaje del usuario, del modelo de chat, para que el modelo de chat observe un comportamiento y de una llamada a una herramienta.


`**ChatPromptTemplate**` simplifica la creación de estos mensajes estructurados, evitando la necesidad de concatenarlos manualmente en una sola cadena de texto.

Creemos una lista de mensajes.


In [13]:
chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Eres un experto en literatura fantastica y ciencia ficción"),
    ("human", "Proporciona un listado con los {numero} autores mas importantes de liteartua fantastica en la actualidad"),
])


llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0)

respuesta = llm.invoke(chat_prompt_template.format_messages(numero=5))

respuesta



AIMessage(content='Aquí tienes un listado con cinco de los autores más importantes de literatura fantástica en la actualidad:\n\n1. **Neil Gaiman**: Conocido por obras como "American Gods", "Coraline" y "The Ocean at the End of the Lane", Gaiman ha sido una figura influyente en la literatura fantástica contemporánea, combinando mitología, folclore y elementos de la cultura pop.\n\n2. **N.K. Jemisin**: Autora de la aclamada trilogía "La Tierra Fragmentada", que incluye "La Quinta Estación", Jemisin ha sido reconocida por su innovador enfoque de la fantasía, así como por su exploración de temas sociales y políticos.\n\n3. **Brandon Sanderson**: Famoso por su serie "El Archivo de las Tormentas" y su trabajo en el universo de "Mistborn", Sanderson es conocido por su construcción de mundos complejos y sistemas de magia bien definidos.\n\n4. **Ursula K. Le Guin**: Aunque falleció en 2018, su influencia perdura en la literatura fantástica. Obras como "La mano izquierda de la oscuridad" y "Los

La respuesta del LLM es un objeto AIMessage.
Habitualmente la parte en la que estamos interesado es content



In [14]:
respuesta.content

'Aquí tienes un listado con cinco de los autores más importantes de literatura fantástica en la actualidad:\n\n1. **Neil Gaiman**: Conocido por obras como "American Gods", "Coraline" y "The Ocean at the End of the Lane", Gaiman ha sido una figura influyente en la literatura fantástica contemporánea, combinando mitología, folclore y elementos de la cultura pop.\n\n2. **N.K. Jemisin**: Autora de la aclamada trilogía "La Tierra Fragmentada", que incluye "La Quinta Estación", Jemisin ha sido reconocida por su innovador enfoque de la fantasía, así como por su exploración de temas sociales y políticos.\n\n3. **Brandon Sanderson**: Famoso por su serie "El Archivo de las Tormentas" y su trabajo en el universo de "Mistborn", Sanderson es conocido por su construcción de mundos complejos y sistemas de magia bien definidos.\n\n4. **Ursula K. Le Guin**: Aunque falleció en 2018, su influencia perdura en la literatura fantástica. Obras como "La mano izquierda de la oscuridad" y "Los desposeídos" cont

Pero la respuesta contiene abundates datos que en ocasiones podrian ser de utilidad

In [15]:
respuesta.response_metadata

{'token_usage': {'completion_tokens': 349,
  'prompt_tokens': 43,
  'total_tokens': 392,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
 'model_name': 'gpt-4o-mini-2024-07-18',
 'system_fingerprint': 'fp_0aa8d3e20b',
 'finish_reason': 'stop',
 'logprobs': None}

Las herramientas son útiles siempre que se desea que un modelo interactúe con sistemas externos.

Los sistemas externos (por ejemplo, las API) a menudo requieren un esquema de entrada o una carga útil particular, en lugar de lenguaje natural.

Cuando vinculamos una API, por ejemplo, como herramienta, le damos al modelo el conocimiento del esquema de entrada requerido.

El modelo elegirá llamar a una herramienta en función de la entrada en lenguaje natural del usuario.

Y devolverá una salida que se ajuste al esquema de la herramienta.

Muchos proveedores de LLM admiten la llamada a herramientas y la interfaz de llamada a herramientas en LangChain es sencilla.

Simplemente puede pasar cualquier función de Python a ChatModel.bind_tools(function).


Muchos proveedores de LLM admiten la llamada de herramientas y la interfaz de llamada de herramientas en LangChain es sencilla.  

https://python.langchain.com/v0.1/docs/integrations/chat/  
https://blog.langchain.dev/improving-core-tool-interfaces-and-docs-in-langchain/

puedes pasar directamente funciones de Python a métodos como .bind_tools(), y el sistema se encarga de inferir automáticamente los esquemas necesarios a partir de las anotaciones de tipo y las cadenas de documentación (docstrings) de las funciones.

Esta simplificación hace que el proceso sea mucho más intuitivo y reduce la cantidad de código repetitivo.

Podemos pasar cualquier función de Python a ChatModel.bind_tools() , lo que permite que las funciones normales de Python se utilicen directamente como herramientas.

Vamos a crear una funcion que multiplica dos numeros y las vamos a pasar como herramienta a nuestro modelo.

In [23]:
def multiplicar(a: int, b: int) -> int:
    """Multiplica a y b.

    Args:
        a: primer int
        b: segundo int
    """
    return a * b

llm_con_herramientas = llm.bind_tools([multiplicar])

Ahora llm_con_herramientas es un nuevo llm con dos herramientas, y si lo invocamos con cierta pregunta, obtendremos como resultado una AIMessage que sin contenido pero que es una llamada a la herramienta.

In [32]:

respuesta = llm_con_herramientas.invoke("Cuanto es 2 por 3")
respuesta


AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_73VIffeOxvXaBdE8EjjITtwN', 'function': {'arguments': '{"a":2,"b":3}', 'name': 'multiplicar'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 92, 'total_tokens': 112, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-e22b4a33-9160-4c7e-82ef-3df7e545b93c-0', tool_calls=[{'name': 'multiplicar', 'args': {'a': 2, 'b': 3}, 'id': 'call_73VIffeOxvXaBdE8EjjITtwN', 'type': 'tool_call'}], usage_metadata={'input_tokens': 92, 'output_tokens': 20, 'total_tokens': 112, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

LangChain analizará las anotaciones de tipo y las cadenas de documentación para inferir los esquemas necesarios.

In [33]:
respuesta.additional_kwargs['tool_calls']

[{'id': 'call_73VIffeOxvXaBdE8EjjITtwN',
  'function': {'arguments': '{"a":2,"b":3}', 'name': 'multiplicar'},
  'type': 'function'}]

Si el prompy no tiene nada que ver no se llamara a la herramienta

In [35]:
respuesta = llm_con_herramientas.invoke("Que es un pájaro ?")
respuesta

AIMessage(content='Un pájaro es un animal vertebrado que pertenece a la clase Aves. Se caracteriza por tener plumas, un pico sin dientes, y la mayoría de las especies son capaces de volar. Los pájaros tienen un esqueleto ligero y un sistema respiratorio eficiente que les permite obtener el oxígeno necesario para el vuelo. \n\nLos pájaros son ovíparos, lo que significa que ponen huevos, y suelen cuidar de sus crías hasta que son lo suficientemente grandes para valerse por sí mismas. Existen miles de especies de pájaros en todo el mundo, que varían en tamaño, color, hábitat y comportamiento. Algunos ejemplos comunes de pájaros son los gorriones, las palomas, los águilas y los pingüinos.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 165, 'prompt_tokens': 91, 'total_tokens': 256, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': 

Veamos un ejemplo solo un poco mas complicado. Vamos a dotar a nuestro modelo de cuatro herramientas de calculo sencillas.

In [42]:
# from langchain_openai import ChatOpenAI
from langchain.tools import tool


def sumar(a: float, b: float) -> float:
    """Suma dos números."""
    return a + b


def restar(a: float, b: float) -> float:
    """Resta dos números."""
    return a - b


def multiplicar(a: float, b: float) -> float:
    """Multiplica dos números."""
    return a * b


def dividir(a: float, b: float) -> float:
    """Divide dos números."""
    if b == 0:
        return "Error: No se puede dividir por cero."
    return a / b

# Crear el modelo de OpenAI
modelo = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0)

# Vincular las herramientas al modelo usando .bind_tools
modelo_con_herramientas = modelo.bind_tools([sumar, restar, multiplicar, dividir])

# Preguntar al modelo
pregunta = "¿Cuánto es 15 - 20? Luego, divide el resultado por 5."
respuesta = modelo_con_herramientas.invoke(pregunta)

# Mostrar la respuesta
print(respuesta.additional_kwargs['tool_calls'])

[{'id': 'call_qt0UhA2gjSiBXFbFtGM5vGXt', 'function': {'arguments': '{"a": 15, "b": 20}', 'name': 'restar'}, 'type': 'function'}, {'id': 'call_7IY5w7WQExYhSEvuAro4ZiQu', 'function': {'arguments': '{"a": 15, "b": 5}', 'name': 'dividir'}, 'type': 'function'}]


Observa como cambiando el prompt obtenemos resultados diferentes. Desde una llamada a una herramienta a un resultado directo.

Hablar del decorador @tool
https://python.langchain.com/v0.2/docs/how_to/custom_tools/?ref=blog.langchain.dev#creating-tools-from-functions



Ahora toca crear un grafo con todo esto y aqui se acaba